In [44]:
def get_digit(index, k, n):
    return (index >> (2 * (n - 1 - k))) & 3

def set_digit(index: int, k: int, new_digit: int, n: int) -> int:
    shift = 2 * (n - 1 - k)
    index &= ~(3 << shift)  # Xóa 2 bit tại vị trí k
    index |= (new_digit << shift)  # Đặt giá trị mới
    return index

def apply_cnot(index, control, target, n):
    char_control = get_digit(index, control, n)
    char_target = get_digit(index, target, n)
    
    lambdas = 1 
    
    if char_control == 0:  # 'i'
        if char_target == 2:  # 'y'
            new_control = 3  # 'z'
            new_target = 2   # 'y'
        elif char_target == 3:  # 'z'
            new_control = 3  # 'z'
            new_target = 3   # 'z'
        else:
            new_control = char_control
            new_target = char_target
    elif char_control == 1:  # 'x'
        if char_target == 0:  # 'i'
            new_control = 1  # 'x'
            new_target = 1   # 'x'
        elif char_target == 1:  # 'x'
            new_control = 1  # 'x'
            new_target = 0   # 'i'
        elif char_target == 2:  # 'y'
            new_control = 2  # 'y'
            new_target = 3   # 'z'
        elif char_target == 3:  # 'z'
            lambdas = -1
            new_control = 2  # 'y'
            new_target = 2   # 'y'
    elif char_control == 2:  # 'y'
        if char_target == 0:  # 'i'
            new_control = 2  # 'y'
            new_target = 1   # 'x'
        elif char_target == 1:  # 'x'
            new_control = 2  # 'y'
            new_target = 2   # 'y'
        elif char_target == 2:  # 'y'
            new_control = 1  # 'x'
            new_target = 3   # 'z'
            lambdas = -1
        elif char_target == 3:  # 'z'
            new_control = 1  # 'x'
            new_target = 2   # 'y'
    elif char_control == 3:  # 'z'
        if char_target == 2:  # 'y'
            new_control = 0  # 'i'
            new_target = 2   # 'y'
        elif char_target == 3:  # 'z'
            new_control = 0  # 'i'
            new_target = 3   # 'z'
        else:
            new_control = char_control
            new_target = char_target
    
    # Cập nhật số nguyên
    new_index = set_digit(index, control, new_control, n)
    new_index = set_digit(new_index, target, new_target, n)
    
    return new_index, lambdas

for i in range(0, 4**12):
	(apply_cnot(i, 0, 3, 10))






In [49]:
import cupy as cp

# Định nghĩa kernel
apply_cnot_kernel = cp.RawKernel(r'''
extern "C" __global__
void apply_cnot(const int* word_int_array, int control, int target, int n, int* result_array, int* lambdas_array, int num_words) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_words) {
        int word_int = word_int_array[idx];
        
        auto get_digit = [](int word_int, int k, int n) {
            return (word_int >> (2 * (n - 1 - k))) & 3;
        };
        
        auto set_digit = [](int word_int, int k, int new_digit, int n) {
            int shift = 2 * (n - 1 - k);
            word_int &= ~(3 << shift);  
            word_int |= (new_digit << shift);  
            return word_int;
        };
        
        int char_control = get_digit(word_int, control, n);
        int char_target = get_digit(word_int, target, n);
        
        int new_control = char_control;
        int new_target = char_target;
        int lambdas = 1;
        
        if (char_control == 0) {  // 'i'
            if (char_target == 2) {  // 'y'
                new_control = 3;  // 'z'
                new_target = 2;   // 'y'
            } else if (char_target == 3) {  // 'z'
                new_control = 3;  // 'z'
                new_target = 3;   // 'z'
            }
        } else if (char_control == 1) {  // 'x'
            if (char_target == 0) {  // 'i'
                new_control = 1;  // 'x'
                new_target = 1;   // 'x'
            } else if (char_target == 1) {  // 'x'
                new_control = 1;  // 'x'
                new_target = 0;   // 'i'
            } else if (char_target == 2) {  // 'y'
                new_control = 2;  // 'y'
                new_target = 3;   // 'z'
            } else if (char_target == 3) {  // 'z'
                lambdas = -1;
                new_control = 2;  // 'y'
                new_target = 2;   // 'y'
            }
        } else if (char_control == 2) {  // 'y'
            if (char_target == 0) {  // 'i'
                new_control = 2;  // 'y'
                new_target = 1;   // 'x'
            } else if (char_target == 1) {  // 'x'
                new_control = 2;  // 'y'
                new_target = 2;   // 'y'
            } else if (char_target == 2) {  // 'y'
                new_control = 1;  // 'x'
                new_target = 3;   // 'z'
                lambdas = -1;
            } else if (char_target == 3) {  // 'z'
                new_control = 1;  // 'x'
                new_target = 2;   // 'y'
            }
        } else if (char_control == 3) {  // 'z'
            if (char_target == 2) {  // 'y'
                new_control = 0;  // 'i'
                new_target = 2;   // 'y'
            } else if (char_target == 3) {  // 'z'
                new_control = 0;  // 'i'
                new_target = 3;   // 'z'
            }
        }
        
        int new_word_int = set_digit(word_int, control, new_control, n);
        new_word_int = set_digit(new_word_int, target, new_target, n);
        
        result_array[idx] = new_word_int;
        lambdas_array[idx] = lambdas;
    }
}
''', 'apply_cnot')

# Ví dụ sử dụng
n = 13  # Số qubit
control = 0  # Vị trí qubit điều khiển
target = 3  # Vị trí qubit mục tiêu
num_words = 4**n # Số lượng word_int trong mảng

# Mảng đầu vào (ví dụ: [4, 2, 7] tương ứng với ["xi", "iy", "xz"])
word_int_array = cp.array(range(0, num_words), dtype=cp.int32)

# Tạo mảng kết quả
result_array = cp.zeros_like(word_int_array)
lambdas_array = cp.zeros(num_words, dtype=cp.int32)

# Gọi kernel
block_size = 256
grid_size = (num_words + block_size - 1) // block_size
apply_cnot_kernel((grid_size,), (block_size,), (word_int_array, control, target, n, result_array, lambdas_array, num_words))

# # In kết quả
# print("Kết quả sau khi áp dụng CNOT:")
# print("new_word_int:", result_array)
# print("lambdas:", lambdas_array)